***Pruebas para la determinación de normalidad en los datos medidos intralaboratorio*** 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Bibliotecas 

Se importan las bibliotecas necesarias para el analisis de los datos 

In [ ]:
!pip install pingouin ## Se instala manualmente esta biblioteca

In [ ]:
import numpy as np ## Manipulación matematica 
import pandas as pd ## Manipulación de conjuntos de datos 
from scipy import stats ## Invocar funciones estadisticas 
from sklearn.linear_model import LinearRegression ## Generar un modelo de regresión lineal 
import matplotlib.pyplot as plt ## Imprimir graficas (Regresión linela)
from statsmodels.stats.stattools import durbin_watson ## Modelo durbin watson para la verificación de independencia de residuos 
import pingouin as pg ## Resultados estadisiticos 
from scipy.stats import mannwhitneyu
from statistics import stdev ## Desviación estandar de una muestra

## Carga de los datos

En esta sección se carga, primero el excel, que contiene los datos intralaboratorio, y se especifica, las mediciones de los dos participantes 

Se cargan los datos a los cuales se pretende realizar las pruebas de normalidad

In [ ]:
def numeric_columns(matriz_data):
    matriz_data_c = matriz_data
    for i in matriz_data_c.columns[1:]:
        matriz_data_c[i] = matriz_data_c[i].astype(float) 
    return matriz_data_c

In [ ]:
comparacion_intra_lab = '/content/drive/MyDrive/Análisis estadístico  Dishegro/Prueba intra-laboratorio/Analisis estadistico/DS05-F04 008  2022 Par torsional.xlsm' ## Se define la dirección en donde se encuentra el documento 
int_lab = pd.read_excel(comparacion_intra_lab, sheet_name = 'DC05-F04 Intralaboratorio') ## con la función read_excel se lee la dirección, la entrada sheet_name nos especifica que hoja se quiere leer 


Se genera la tabla que almacena los puntos de medicón con sus respectivos operarios

In [ ]:
puntos_de_medición = pd.DataFrame(int_lab.iloc[25:32,0:4]) ## definimos la tabla que contiene las mediciones a analizar 
puntos_de_medición.columns = ['Nombre','1 punto de medición','2 punto de medición','3 punto de medición'] ## Se convierten en Dataframe para manipular mejor y se le agrega nombre a cada columna

In [ ]:
puntos_de_medición = puntos_de_medición.reset_index()## Resetea el indice para que no quede con el indice del Excel
puntos_de_medición = puntos_de_medición.drop([0],axis = 0) ## Borra las filas del indice anterior 
puntos_de_medición = puntos_de_medición.drop(['index'],axis = 1) ## Borra la columna de indice 

Se realiza un dataframe de las mediciones por los dos operarios 

In [ ]:
puntos_de_medición.iloc[:3,0] = 'Operario 1' ## Definir las mediciones del operario 1
puntos_de_medición.iloc[3:,0] = 'Operario 2' ## Definir las mediciones del operario2 
puntos_de_medición

De los puntos de medida, se definen los dos operarios implicado en la prueba, en caso de requerir, las dos muestras por separado 


In [ ]:
operario_1 = puntos_de_medición.loc[puntos_de_medición['Nombre'] == 'Operario 1'] ## Vector con las muestras del operario 1
operario_2 = puntos_de_medición.loc[puntos_de_medición['Nombre'] == 'Operario 2'] ## Vector con las muestras del operario 2

A pesar de definir especificamente cada la matriz para cada vector, considero que este proceso se puede realizar de forma mas automatizada, definiendo bien las posibles entradas, en otros documentos de excel 

## Funciones necesarias para realizar las pruebas de normalidad 

### Prueba de normalidad Shapiro Wilk 

Objetivo: La función tiene como objetivo, realizar la prueba en un Dataframe, cuyas columnas tengan los datos correspondientes a un punto de medición, cada una

Entrada: La entrada para la función es la matriz puntos de medición 

Procedimiento: Se crea una lista vacia, llamada sha_test, en esta lista se anexaran los resultados obtenidos de aplicar la prueba en cada punto.
Para aplicar la prueba en cada punto, se utiliza un ciclo for (list comprehension), que aplica la prueba en cada columna de la matriz, posterior a esto se crea un dataFrame con los resultados, devolviendo dicha lista  

Salida: Un Dataframe con los resultados estadisticos y p-valor para cada uno de los puntos de la matriz, resultado de aplicar Shapiro Wilk

Interpretación: El test de Shapiro-Wilks plantea la hipótesis nula de que una muestra proviene de una distribución normal. Eligimos un nivel de significanza, por ejemplo 0,05, y tenemos una hipótesis alternativa que sostiene que la distribución no es normal.

En conclusión si el p-valor de la prueba de Shapiro-Wilk es superior a 0,05, los datos son normales. Si está por debajo de 0,05, los datos se desvían significativamente de una distribución normal.

In [ ]:
def shapiro_test(pts_med):
    sha_test = []
    sha_test = ([stats.shapiro(pts_med[i]) for i   in  pts_med.columns[1:]])
    sha_test  = pd.DataFrame(sha_test, index = pts_med.columns[1:])
    return sha_test

### Prueba de homoscedasticidad (Levene test)

Objetivo: La función tiene como objetivo, realizar la prueba de comparación entre dos Dataframes, cuyas columnas tengan los datos correspondientes a un punto de medición

Entrada: La entrada para la función son dos vectores con los puntos medidos para cada operario

Procedimiento: Se crea una lista vacia, llamada lev_test, en esta lista se anexaran los resultados obtenidos al comparar lo medido por cada operario por medio de la prueba en cada punto. Para aplicar la prueba en cada punto, se utiliza un ciclo for (list comprehension), que aplica la prueba en cada columna de la matriz, posterior a esto se crea un dataFrame con los resultados, devolviendo dicha lista

La función stats.levene, implica la especificación del estadistico a comparar, definida en la cariable center, dicha prueba tiene tres variaciones, explicadas a continuacion:

on posibles tres variaciones de la prueba de Levene. Las posibilidades y sus usos recomendados son:

'median' : Recomendado para distribuciones sesgadas (no normales)

'mean' : Recomendado para distribuciones simétricas de cola moderada.

'trimmed' : Recomendado para distribuciones de colas pesadas.

La versión de prueba que utiliza la media fue propuesta en el artículo original de Levene ( *Levene, H. (1960). En Contribuciones a la probabilidad y la estadística: ensayos en honor de Harold Hotelling, I. Olkin et al. eds., Stanford University Press, págs. 278-292.*), mientras que la mediana y la media recortada han sido estudiadas por Brown y Forsythe (*Brown, MB y Forsythe, AB (1974), Revista de la Asociación Estadounidense de Estadística, 69, 364-367*), a veces también denominada prueba de Brown-Forsythe.

En este caso se utiliza, el valor pre-determinado 'median', puesto que existen indicios de que la prueba no es normal de acuerdo a lo anteriormente obtenido en la función de shapiro wilk

Salida: Un Dataframe con los resultados estadisticos y p-valor para cada uno de los puntos de la matriz, resultado de aplicar Levene test

La prueba se usa para probar si k muestras tienen varianzas iguales. La igualdad de varianzas entre muestras se denomina homogeneidad de varianza. Algunas pruebas estadísticas, por ejemplo, el análisis de varianza, suponen que las varianzas son iguales entre grupos o muestras. La prueba de Levene se puede utilizar para verificar esa suposición.

Se considera como hipótesis nula que los datos proceden de distribuciones con la misma varianza (homocedasticidad). ***Por lo tanto, si el p-value es menor que un determinado valor (típicamente 0.05), entonces se considera que hay evidencias suficientes para rechazar la homocedasticidad en favor de la heterocedasticidad.***

In [ ]:
def levene_test(pts_med1,pts_med2):
    lev_test = []
    lev_test = [stats.levene(pts_med1[i], pts_med2[i], center='median') for i in   operario_2.columns[1:]]
    lev_test  = pd.DataFrame(lev_test, index = pts_med1.columns[1:])
    return lev_test

### Independencia de residuos 

Objetivo: La función tiene como objetivo, realizar y graficar la regresión lineal para cada una de los puntos de medición, posteriormente obtener los residuos y aplicar prueba de independencia sobre estos 

Entrada: La entrada para la función es la matriz puntos de medición, esto en caso de la función que permite el ciclo for, sin embargo para la función regresión linel, la entrada es las muestras para cada punto (las columnas de la matrix)

Procedimiento: Se crea un vector x, que enumerara los puntos de cada muestra, se prepara luego el modelo de función lineal, para posteriormente involucrar dicha x y los puntos de medición, con el fin de predecir unos puntos, producto del modelo de refresion lineal, finalemnte se restan los puntos originales y los obtenidos por el modelo, para asi tener los residuos, dichos residuos se le aplica la función durbin_watson, para obtener el estadistico correspondiente 

Salida, se obtiene un dataFrame con el estadistico de Durbin Watson en cada punto

Interpretación: La prueba de Durbin-Watson, se utiliza para detectar la presencia de autocorrelación en los residuos de una regresión, en otras palabras para corroborar independencia de residuos 


La estadística de prueba siempre varía de 0 a 4 donde:

* d = 2 indica que no hay autocorrelación
* d <2 indica correlación serial positiva
* d > 2 indica correlación serial negativa

In [ ]:
def regresión_lineal(pt_med,i):
    x = np.array(range(len(pt_med)))
    regresion_lineal = LinearRegression()
    regresion_lineal.fit(x.reshape(-1, 1), pt_med)
    pt_med_prediccion = regresion_lineal.predict(x.reshape(-1,1))
    residuos = pt_med - pt_med_prediccion

    ## Grafica puntos comparados con la regresión lienal 
    plt.scatter(x,pt_med,label='Puntos medidos', color='blue')
    plt.plot(x,pt_med_prediccion,label='Modelos estadistico', color='red',)
    plt.title(i);
    plt.ylim(0,10)
    plt.show()

    ## Independencia de residuos 
    print('El valor del estadistico de Durbin Watson es: ', durbin_watson(residuos))
    Prueba_independencia = durbin_watson(residuos)
    return Prueba_independencia

In [ ]:
def ciclo_rl(pts_med):
    independencia = []
    for i in pts_med.columns[1:]:
        independencia.append(regresión_lineal(pts_med[i],i))

    independencia  = pd.DataFrame(independencia, index = pts_med.columns[1:],columns= ['Estadistico Durbin Watson'])
    
    return independencia

## Criterio aplicar analisis de varianza parametrico o no parametrico 

In [ ]:
def fila_criterios(s_test,l_test,i_e):
    criterion = []
    criterion.append(s_test >= 0.05)
    criterion.append(l_test >= 0.05)
    criterion.append(i_e == 2)
    return criterion

In [ ]:
def criterio(pts_med,pts_med1,pts_med2):
    sha_test = shapiro_test(pts_med)
    lev_test = levene_test(pts_med1,pts_med2)
    ind_re = ciclo_rl(pts_med)
    matriz_criterion = []
    values_matriz = []
    for i in pts_med.columns[1:]:
      values_matriz.append([sha_test.loc[i,'pvalue'],
                                              lev_test.loc[i,'pvalue'],
                                               ind_re.loc[i,'Estadistico Durbin Watson']])
      matriz_criterion.append(fila_criterios(sha_test.loc[i,'pvalue'],
                                              lev_test.loc[i,'pvalue'],
                                               ind_re.loc[i,'Estadistico Durbin Watson']))
    
    matriz_criterion  = pd.DataFrame(matriz_criterion, index = pts_med.columns[1:],columns=['Shapiro-Wilk','Levene test','Durbin Watson'])
    values_matriz  = pd.DataFrame(values_matriz, index = pts_med.columns[1:],columns=['Shapiro-Wilk','Levene test','Durbin Watson'])
    print(matriz_criterion)
    print(values_matriz)
    return matriz_criterion,values_matriz

In [ ]:
def anova_test(pts_med,i):
    puntos_de_medición_numeric = numeric_columns(pts_med)
    anova_results = []
    results = pg.anova(data=puntos_de_medición_numeric, dv= i, between='Nombre', detailed=True)

    
    ## Calculo de F-critico 

    # F critical 
    alpha = 0.05

    F_critico = stats.f.ppf(1-alpha, results['DF'][0], results['DF'][1])
    results['F crit']= F_critico
    
    print(results)
    anova_results.append(results)

    return anova_results

In [ ]:
def mannwhitneyu_test(pts_med1,pts_med2,i):
    mannwhitneyu_results = []
    aux = mannwhitneyu(pts_med1[i], pts_med2[i], method="exact") 
    print(aux)
    mannwhitneyu_results.append(aux)
    return mannwhitneyu_results

In [ ]:
def p_or_np(boolean_c,pts_med,pts_med1,pts_med2):
  for i in boolean_c.index[:] :
      aux = boolean_c.loc[i,:].value_counts().index[0]
      if (aux == True):
        print('Resultado para:' ,i)
        anova_result = anova_test(pts_med,i)
      else:
        print('Resultado para:' ,i)
        mannwhitneyu_results = mannwhitneyu_test(pts_med1,pts_med2,i)
        

  return anova_result,mannwhitneyu_results

## Prueba h y k Mandel 

In [ ]:
puntos_de_medición

Para aplicar la prueba de h y k de mendel se desea, obtener el numero de participantes en la prueba, por tal motivo, aplicamos la función unique(), dicha función nos permite obtener los valores unicos de una columna, en este caso será la columna nombres pues esta contiene los participantes en la prueba, dicho tamaño de vector resultante lo definiremos con la variable 'p', para cumplir con la simbologia matematica

In [ ]:
nombres_p = puntos_de_medición['Nombre'].unique()
p = len(nombres_p)

In [ ]:
mean_muestra = np.zeros((p, len(puntos_de_medición.columns[1:])))

i_n = 0 
for i in puntos_de_medición.columns[1:]:
    j_n = 0 
    for j in nombres_p:
        
        x_aux = puntos_de_medición[i].loc[puntos_de_medición['Nombre']== j] 
        mean_x = np.mean(x_aux)
        mean_muestra[j_n,i_n] = mean_x
        j_n += 1

    i_n += 1


        

In [ ]:
std_muestra = np.zeros((p, len(puntos_de_medición.columns[1:])))

i_n = 0 
for i in puntos_de_medición.columns[1:]:
    j_n = 0 
    for j in nombres_p:
        
        x_aux = puntos_de_medición[i].loc[puntos_de_medición['Nombre']== j] 
        std_x = stdev(x_aux)
        std_muestra[j_n,i_n] = std_x
        j_n += 1

    i_n += 1

In [ ]:
std_muestra = pd.DataFrame(std_muestra, 
                            index = [nombres_p],
                            columns = [puntos_de_medición.columns[1:]])

In [ ]:
mean_muestra = pd.DataFrame(mean_muestra, 
                            index = [nombres_p],
                            columns = [puntos_de_medición.columns[1:]])


In [ ]:
mean_muestra

In [ ]:
std_muestra

In [ ]:
h_matriz = mean_muestra.copy()
for i in mean_muestra.columns:
    x_ = np.mean(mean_muestra[i])
    s = list(mean_muestra[i])
    s = stdev(s)
    for j in mean_muestra.index:
        x = mean_muestra[i].loc[j]
        h = (x-x_) / (s)
        h_matriz[i].loc[j] = h      

In [ ]:
h_matriz

In [ ]:
k_matriz = std_muestra.copy()
for i in std_muestra.columns:
    s_= np.sqrt((np.sum((std_muestra[i])**2))/ p)
    for j in std_muestra.index:
      si = std_muestra[i].loc[j]
      k = si/s_
      k_matriz[i].loc[j] = k

In [ ]:
k_matriz

In [ ]:
# Import Library
import scipy.stats
  
# To find the T critical value
t_crit = scipy.stats.t.ppf(q=.05/2,df=(p-1))
# To find the F critical value 
f_crit = scipy.stats.f.ppf(q=1-.01, dfn=2-1, dfd=8-1)
print(f_crit)

In [ ]:
h_crit = ((p-1)*(t_crit)) /(np.sqrt((p)*((t_crit**2)+p-2)))

In [ ]:
h_crit

## Grubbs test 

In [ ]:
!pip install outlier_utils

In [ ]:
from outliers import smirnov_grubbs as grubbs
data = list(puntos_de_medición['3 punto de medición'])
print(data)
grubbs.test(data, alpha=.01)